In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Cystic_Fibrosis"
cohort = "GSE67698"

# Input paths
in_trait_dir = "../../input/GEO/Cystic_Fibrosis"
in_cohort_dir = "../../input/GEO/Cystic_Fibrosis/GSE67698"

# Output paths
out_data_file = "../../output/preprocess/Cystic_Fibrosis/GSE67698.csv"
out_gene_data_file = "../../output/preprocess/Cystic_Fibrosis/gene_data/GSE67698.csv"
out_clinical_data_file = "../../output/preprocess/Cystic_Fibrosis/clinical_data/GSE67698.csv"
json_path = "../../output/preprocess/Cystic_Fibrosis/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Transcriptional profiling of CFBE41o-ΔF508 cells and CFBE41o−CFTR cells."
!Series_summary	"Cystic fibrosis bronchial epithelial (CFBE41o-ΔF508) cells subjected to 23 bio-active small molecules including vehicle controls, at low temperature and untreated cells. Untreated Cystic fibrosis bronchial epithelial cells (CFBE41o−CFTR) are also included."
!Series_overall_design	"Two-colors Dye-swap, two or three biological replicates with two technical replicates each and DMSO (vehicle control) were included in each hybridization batch."
Sample Characteristics Dictionary:
{0: ['cell line: polarized CFBE41o-cell line'], 1: ['transduction: TranzVector lentivectors containing deltaF508 CFTR (CFBE41o-deltaF508CFTR)', 'transduction: TranzVector lentivectors containing wildtype CFTR (CFBE41o-CFTR)']}


### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
import os
import pandas as pd
import json
from typing import Callable, Optional, Dict, Any

# 1. Assess gene expression data availability
# Based on the background information, this dataset contains transcriptional profiling data
# which indicates gene expression data is available
is_gene_available = True

# 2.1 Identify rows for trait, age, and gender
# From the sample characteristics, we can see information about cell line and transduction type
# The transduction type (row 1) indicates whether cells have deltaF508 CFTR mutation or wildtype CFTR
# This can be used as our trait (presence of cystic fibrosis mutation)
trait_row = 1
# Age and gender information are not available in this dataset
age_row = None
gender_row = None

# 2.2 Define conversion functions
def convert_trait(value):
    """
    Convert CF status based on transduction information
    deltaF508 CFTR = 1 (CF mutation present)
    wildtype CFTR = 0 (No CF mutation)
    """
    if value is None:
        return None
        
    if isinstance(value, str) and ":" in value:
        value = value.split(":", 1)[1].strip()
    
    if isinstance(value, str):
        if "deltaF508" in value:
            return 1  # CF mutation
        elif "wildtype CFTR" in value:
            return 0  # No CF mutation
    
    return None

def convert_age(value):
    # Not applicable for this dataset
    return None

def convert_gender(value):
    # Not applicable for this dataset
    return None

# 3. Save metadata - perform initial filtering
# Check if trait data is available (trait_row is not None)
is_trait_available = trait_row is not None

# Initial validation to check if dataset passes basic requirements
validate_and_save_cohort_info(
    is_final=False, 
    cohort=cohort, 
    info_path=json_path, 
    is_gene_available=is_gene_available, 
    is_trait_available=is_trait_available
)

# 4. Extract clinical features if trait data is available
if trait_row is not None:
    # Make sure the output directory exists
    os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
    
    # Access the clinical data that was loaded in a previous step
    # This is typically a DataFrame with samples as rows and characteristic indices as columns
    # We'll assume clinical_data exists from previous steps
    
    # Extract clinical features
    selected_clinical_df = geo_select_clinical_features(
        clinical_df=clinical_data,  # Use existing clinical_data from previous step
        trait=trait,
        trait_row=trait_row,
        convert_trait=convert_trait,
        age_row=age_row,
        convert_age=convert_age,
        gender_row=gender_row,
        convert_gender=convert_gender
    )
    
    # Preview the selected clinical features
    preview = preview_df(selected_clinical_df)
    print("Preview of selected clinical features:")
    print(preview)
    
    # Save the clinical data to CSV
    selected_clinical_df.to_csv(out_clinical_data_file)
    print(f"Clinical data saved to {out_clinical_data_file}")


Preview of selected clinical features:
{'GSM1946634': [1.0], 'GSM1946635': [1.0], 'GSM1946636': [1.0], 'GSM1946637': [1.0], 'GSM1946638': [1.0], 'GSM1946639': [1.0], 'GSM1946640': [1.0], 'GSM1946641': [1.0], 'GSM1946642': [1.0], 'GSM1946643': [1.0], 'GSM1946644': [1.0], 'GSM1946645': [1.0], 'GSM1946646': [1.0], 'GSM1946647': [1.0], 'GSM1946648': [1.0], 'GSM1946649': [1.0], 'GSM1946650': [1.0], 'GSM1946651': [1.0], 'GSM1946652': [1.0], 'GSM1946653': [1.0], 'GSM1946654': [1.0], 'GSM1946655': [1.0], 'GSM1946656': [1.0], 'GSM1946657': [1.0], 'GSM1946658': [1.0], 'GSM1946659': [1.0], 'GSM1946660': [1.0], 'GSM1946661': [1.0], 'GSM1946662': [1.0], 'GSM1946663': [1.0], 'GSM1946664': [1.0], 'GSM1946665': [1.0], 'GSM1946666': [1.0], 'GSM1946667': [1.0], 'GSM1946668': [1.0], 'GSM1946669': [1.0], 'GSM1946670': [1.0], 'GSM1946671': [1.0], 'GSM1946672': [1.0], 'GSM1946673': [1.0], 'GSM1946674': [1.0], 'GSM1946675': [1.0], 'GSM1946676': [1.0], 'GSM1946677': [1.0], 'GSM1946678': [1.0], 'GSM1946679': [

### Step 3: Gene Data Extraction

In [4]:
# 1. Get the file paths for the SOFT file and matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. First, let's examine the structure of the matrix file to understand its format
import gzip

# Peek at the first few lines of the file to understand its structure
with gzip.open(matrix_file, 'rt') as file:
    # Read first 100 lines to find the header structure
    for i, line in enumerate(file):
        if '!series_matrix_table_begin' in line:
            print(f"Found data marker at line {i}")
            # Read the next line which should be the header
            header_line = next(file)
            print(f"Header line: {header_line.strip()}")
            # And the first data line
            first_data_line = next(file)
            print(f"First data line: {first_data_line.strip()}")
            break
        if i > 100:  # Limit search to first 100 lines
            print("Matrix table marker not found in first 100 lines")
            break

# 3. Now try to get the genetic data with better error handling
try:
    gene_data = get_genetic_data(matrix_file)
    print(gene_data.index[:20])
except KeyError as e:
    print(f"KeyError: {e}")
    
    # Alternative approach: manually extract the data
    print("\nTrying alternative approach to read the gene data:")
    with gzip.open(matrix_file, 'rt') as file:
        # Find the start of the data
        for line in file:
            if '!series_matrix_table_begin' in line:
                break
                
        # Read the headers and data
        import pandas as pd
        df = pd.read_csv(file, sep='\t', index_col=0)
        print(f"Column names: {df.columns[:5]}")
        print(f"First 20 row IDs: {df.index[:20]}")
        gene_data = df


Found data marker at line 69
Header line: "ID_REF"	"GSM1946634"	"GSM1946635"	"GSM1946636"	"GSM1946637"	"GSM1946638"	"GSM1946639"	"GSM1946640"	"GSM1946641"	"GSM1946642"	"GSM1946643"	"GSM1946644"	"GSM1946645"	"GSM1946646"	"GSM1946647"	"GSM1946648"	"GSM1946649"	"GSM1946650"	"GSM1946651"	"GSM1946652"	"GSM1946653"	"GSM1946654"	"GSM1946655"	"GSM1946656"	"GSM1946657"	"GSM1946658"	"GSM1946659"	"GSM1946660"	"GSM1946661"	"GSM1946662"	"GSM1946663"	"GSM1946664"	"GSM1946665"	"GSM1946666"	"GSM1946667"	"GSM1946668"	"GSM1946669"	"GSM1946670"	"GSM1946671"	"GSM1946672"	"GSM1946673"	"GSM1946674"	"GSM1946675"	"GSM1946676"	"GSM1946677"	"GSM1946678"	"GSM1946679"	"GSM1946680"	"GSM1946681"	"GSM1946682"	"GSM1946683"	"GSM1946684"	"GSM1946685"	"GSM1946686"	"GSM1946687"	"GSM1946688"	"GSM1946689"	"GSM1946690"	"GSM1946691"	"GSM1946692"	"GSM1946693"	"GSM1946694"	"GSM1946695"	"GSM1946696"	"GSM1946697"	"GSM1946698"	"GSM1946699"	"GSM1946700"	"GSM1946701"	"GSM1946702"	"GSM1946703"	"GSM1946704"	"GSM1946705"	"GSM1946706"	

Index(['A_23_P100001', 'A_23_P100011', 'A_23_P100022', 'A_23_P100056',
       'A_23_P100074', 'A_23_P100092', 'A_23_P100103', 'A_23_P100111',
       'A_23_P100127', 'A_23_P100133', 'A_23_P100141', 'A_23_P100156',
       'A_23_P100177', 'A_23_P100189', 'A_23_P100196', 'A_23_P100203',
       'A_23_P100220', 'A_23_P100240', 'A_23_P10025', 'A_23_P100263'],
      dtype='object', name='ID')


### Step 4: Gene Identifier Review

In [5]:
# Based on the gene identifiers, these appear to be Agilent microarray probe IDs (starting with A_23_P),
# not standard human gene symbols. These will need to be mapped to official gene symbols.

requires_gene_mapping = True


### Step 5: Gene Annotation

In [6]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the SOFT file.
gene_annotation = get_gene_annotation(soft_file)

# 2. Use the 'preview_df' function from the library to preview the data and print out the results.
print("Gene annotation preview:")
print(preview_df(gene_annotation))


Gene annotation preview:
{'ID': ['A_23_P100001', 'A_23_P100011', 'A_23_P100022', 'A_23_P100056', 'A_23_P100074'], 'SPOT_ID': ['A_23_P100001', 'A_23_P100011', 'A_23_P100022', 'A_23_P100056', 'A_23_P100074'], 'CONTROL_TYPE': ['FALSE', 'FALSE', 'FALSE', 'FALSE', 'FALSE'], 'REFSEQ': ['NM_207446', 'NM_005829', 'NM_014848', 'NM_194272', 'NM_020371'], 'GB_ACC': ['NM_207446', 'NM_005829', 'NM_014848', 'NM_194272', 'NM_020371'], 'GENE': [400451.0, 10239.0, 9899.0, 348093.0, 57099.0], 'GENE_SYMBOL': ['FAM174B', 'AP3S2', 'SV2B', 'RBPMS2', 'AVEN'], 'GENE_NAME': ['family with sequence similarity 174, member B', 'adaptor-related protein complex 3, sigma 2 subunit', 'synaptic vesicle glycoprotein 2B', 'RNA binding protein with multiple splicing 2', 'apoptosis, caspase activation inhibitor'], 'UNIGENE_ID': ['Hs.27373', 'Hs.632161', 'Hs.21754', 'Hs.436518', 'Hs.555966'], 'ENSEMBL_ID': ['ENST00000557398', nan, 'ENST00000557410', 'ENST00000300069', 'ENST00000306730'], 'TIGR_ID': [nan, nan, nan, nan, nan]

### Step 6: Gene Identifier Mapping

In [7]:
# 1. Identify which columns to use for mapping
# From the preview, we can see that the 'ID' column contains the probe IDs (A_23_P...)
# and 'GENE_SYMBOL' contains the gene symbols we want to map to
probe_col = 'ID'  # Column with probe identifiers
gene_col = 'GENE_SYMBOL'  # Column with gene symbols

# 2. Get gene mapping dataframe by extracting the probe and gene symbol columns
mapping_df = get_gene_mapping(gene_annotation, probe_col, gene_col)

# 3. Convert probe-level measurements to gene expression data
# This will handle the many-to-many mapping between probes and genes
gene_data = apply_gene_mapping(gene_data, mapping_df)

# Print preview of the mapped gene expression data
print("Gene expression data (after mapping):")
print(f"Shape: {gene_data.shape}")
print(f"First 5 gene symbols: {gene_data.index[:5].tolist()}")
print(f"First 5 sample columns: {gene_data.columns[:5].tolist()}")

# Save the gene expression data
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
gene_data.to_csv(out_gene_data_file)
print(f"Gene expression data saved to {out_gene_data_file}")


Gene expression data (after mapping):
Shape: (18488, 122)
First 5 gene symbols: ['A1BG', 'A1BG-AS1', 'A1CF', 'A2LD1', 'A2M']
First 5 sample columns: ['GSM1946634', 'GSM1946635', 'GSM1946636', 'GSM1946637', 'GSM1946638']


Gene expression data saved to ../../output/preprocess/Cystic_Fibrosis/gene_data/GSE67698.csv


### Step 7: Data Normalization and Linking

In [8]:
# 1. Normalize gene symbols in the gene expression data
normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
print(f"Normalized gene data shape: {normalized_gene_data.shape}")
print("First few genes with their expression values after normalization:")
print(normalized_gene_data.head())

# Save the normalized gene data
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
normalized_gene_data.to_csv(out_gene_data_file)
print(f"Normalized gene data saved to {out_gene_data_file}")

# 2. Link the clinical and genetic data
# Use the clinical data that was already processed and saved in previous steps
selected_clinical_df = pd.read_csv(out_clinical_data_file, index_col=0)
print("Clinical data preview:")
print(preview_df(selected_clinical_df))

# Link the clinical and genetic data
linked_data = geo_link_clinical_genetic_data(selected_clinical_df, normalized_gene_data)
print(f"Linked data shape (before handling missing values): {linked_data.shape}")

# 3. Handle missing values in the linked data
linked_data = handle_missing_values(linked_data, trait)
print(f"Data after handling missing values: {linked_data.shape}")

# 4. Determine whether the trait and demographic features are biased
# First check if trait is biased
trait_type = 'binary' if len(linked_data[trait].unique()) == 2 else 'continuous'
if trait_type == "binary":
    is_trait_biased = judge_binary_variable_biased(linked_data, trait)
else:
    is_trait_biased = judge_continuous_variable_biased(linked_data, trait)

# Check for and remove biased demographic features
unbiased_linked_data = linked_data.copy()
if "Age" in unbiased_linked_data.columns:
    age_biased = judge_continuous_variable_biased(unbiased_linked_data, 'Age')
    if age_biased:
        print(f"The distribution of the feature \'Age\' in this dataset is severely biased.\n")
        unbiased_linked_data = unbiased_linked_data.drop(columns='Age')
    else:
        print(f"The distribution of the feature \'Age\' in this dataset is fine.\n")

if "Gender" in unbiased_linked_data.columns:
    gender_biased = judge_binary_variable_biased(unbiased_linked_data, 'Gender')
    if gender_biased:
        print(f"The distribution of the feature \'Gender\' in this dataset is severely biased.\n")
        unbiased_linked_data = unbiased_linked_data.drop(columns='Gender')
    else:
        print(f"The distribution of the feature \'Gender\' in this dataset is fine.\n")

# 5. Conduct final quality validation and save cohort information
is_usable = validate_and_save_cohort_info(
    is_final=True, 
    cohort=cohort, 
    info_path=json_path, 
    is_gene_available=True, 
    is_trait_available=True, 
    is_biased=is_trait_biased, 
    df=unbiased_linked_data,
    note=f"Dataset contains gene expression data comparing CFBE41o-ΔF508 (CF) cells with CFBE41o−CFTR (control) cells."
)

# 6. If the linked data is usable, save it
if is_usable:
    os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
    unbiased_linked_data.to_csv(out_data_file)
    print(f"Linked data saved to {out_data_file}")
else:
    print("Data was determined to be unusable and was not saved")

Normalized gene data shape: (18247, 122)
First few genes with their expression values after normalization:
          GSM1946634  GSM1946635  GSM1946636  GSM1946637  GSM1946638  \
Gene                                                                   
A1BG          0.7790      0.3736     -0.7298     -0.5658      0.7993   
A1BG-AS1      1.4335      1.1881      0.9390      0.7998      1.9718   
A1CF         -8.6711     -8.6980     -8.1731     -9.8930     -6.6285   
A2M          -5.8238     -5.7392     -5.8264     -7.6944     -5.6462   
A2ML1        -0.9369     -0.8575     -0.6521     -0.4720      2.5128   

          GSM1946639  GSM1946640  GSM1946641  GSM1946642  GSM1946643  ...  \
Gene                                                                  ...   
A1BG         -0.4768     -1.2291     -1.4813      0.8968      0.5161  ...   
A1BG-AS1      1.2580      0.9186      0.8031      1.7652      1.5084  ...   
A1CF         -3.9316    -10.1926    -10.9090     -7.0169     -7.6412  ...   
A2M

Normalized gene data saved to ../../output/preprocess/Cystic_Fibrosis/gene_data/GSE67698.csv
Clinical data preview:
{'GSM1946634': [1.0], 'GSM1946635': [1.0], 'GSM1946636': [1.0], 'GSM1946637': [1.0], 'GSM1946638': [1.0], 'GSM1946639': [1.0], 'GSM1946640': [1.0], 'GSM1946641': [1.0], 'GSM1946642': [1.0], 'GSM1946643': [1.0], 'GSM1946644': [1.0], 'GSM1946645': [1.0], 'GSM1946646': [1.0], 'GSM1946647': [1.0], 'GSM1946648': [1.0], 'GSM1946649': [1.0], 'GSM1946650': [1.0], 'GSM1946651': [1.0], 'GSM1946652': [1.0], 'GSM1946653': [1.0], 'GSM1946654': [1.0], 'GSM1946655': [1.0], 'GSM1946656': [1.0], 'GSM1946657': [1.0], 'GSM1946658': [1.0], 'GSM1946659': [1.0], 'GSM1946660': [1.0], 'GSM1946661': [1.0], 'GSM1946662': [1.0], 'GSM1946663': [1.0], 'GSM1946664': [1.0], 'GSM1946665': [1.0], 'GSM1946666': [1.0], 'GSM1946667': [1.0], 'GSM1946668': [1.0], 'GSM1946669': [1.0], 'GSM1946670': [1.0], 'GSM1946671': [1.0], 'GSM1946672': [1.0], 'GSM1946673': [1.0], 'GSM1946674': [1.0], 'GSM1946675': [1.0], '

Data after handling missing values: (122, 18248)
For the feature 'Cystic_Fibrosis', the least common label is '0.0' with 4 occurrences. This represents 3.28% of the dataset.
Data was determined to be unusable and was not saved
